**Name: XXXX    Student ID: pxxxxxx**

In [ ]:
#Import Python libaries
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation

In [ ]:
#Load movielens data

#Users
users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('/content/u.user', sep='|', names=users_cols, parse_dates=True) 
users

In [ ]:
#Movies
movie_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('/content/u.item', sep='|', names=movie_cols, usecols=range(5),encoding='latin-1')
movies

In [ ]:
#Ratings
rating_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('/content/u.data', sep='\t', names=rating_cols)
ratings

In [ ]:
#Merging movie data with their ratings
movie_ratings = pd.merge(movies, ratings)
movie_ratings

In [ ]:
#merging movie_ratings data with the User's dataframe
df = pd.merge(movie_ratings, users)
df

In [ ]:
#pre-processing
#dropping colums that aren't needed
df.drop(columns=['video_release_date','imdb_url','unix_timestamp'], axis=1, inplace=True)
df

In [ ]:
ratings.drop(columns=['unix_timestamp'], inplace = True, axis = 1 ) 
ratings

In [ ]:
movies.drop(columns=['video_release_date','imdb_url'], inplace = True, axis = 1 )
movies

In [ ]:
#Pivot Table(This creates a matrix of users and movie_ratings)
ratings_matrix = ratings.pivot_table(index=['movie_id'],columns=['user_id'],values='rating').reset_index(drop=True)
ratings_matrix.fillna( 0, inplace = True )
ratings_matrix

In [ ]:
#Pearson Correlation Similarity (Creates a matrix of similaraties ..... which is the pairwise distances
# between two items )

movie_similarity = 1 - pairwise_distances( ratings_matrix, metric="correlation" )
np.fill_diagonal( movie_similarity, 0 ) 
ratings_matrix = pd.DataFrame( movie_similarity )
ratings_matrix

In [ ]:
#Recommender
try:
    #user_inp=input('Enter the reference movie title based on which recommendations are to be made: ')
    user_inp="Mission: Impossible (1996)"
    inp=movies[movies['title']==user_inp].index.tolist()
    inp=inp[0]
    
    movies['similarity'] = ratings_matrix.iloc[inp]
    
except:
    print("No such movies is found!")
    
print("We recommend the following 5 movies based on your choice of ",user_inp ,": \n", movies.sort_values( ["similarity"], ascending = False )[1:6])

#Collaborative Filtering Performance Evaluation

In [ ]:
!pip install scikit-surprise

In [ ]:
#Import Python libraries
import pandas as pd
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import accuracy

In [ ]:
#Load Movieslen dataset
data = Dataset.load_builtin('ml-100k')

In [ ]:
data.raw_ratings

In [ ]:
train_set, test_set = train_test_split(data, test_size=0.30)

K = [10, 20, 50, 100, 200]

result = pd.DataFrame(index=['10','20','50','100','200'],columns=['user-based'])

for x in K:
    algo = KNNWithMeans(k=x, sim_options={
        "name": "pearson",
      "user_based": True,
    })

    algo.fit(train_set)
    predictions = algo.test(test_set)

    result.at[str(x), 'user-based'] = accuracy.rmse(predictions, verbose=True)

result.plot()

**Question 1: Add item-based collaborative filtering using the previous code block.**

**Question 2: Which one performs better? user-based or item-based CF? Why?**

Answer 2: